<a href="https://colab.research.google.com/github/bedoshady1/dataengproject/blob/sara/Data_eng_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# Farida: Data Acquisition & Environment Setup
# Responsibilities: Imports, Loading Data, Removing Duplicates, Date Parsing
# ==============================================================================

import sys, os, warnings
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# 1. Setup & Config
warnings.filterwarnings("ignore")
DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)
pd.options.display.max_columns = 50

# NYC Open Data URLs [cite: 15, 19]
CRASHES_URL = "https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=download"
PERSONS_URL = "https://data.cityofnewyork.us/api/views/f55k-p6yu/rows.csv?accessType=download"

print("👤 MEMBER 1: Loading datasets...")
# Loading with low_memory=False to handle mixed types initially
df_crashes = pd.read_csv(CRASHES_URL, low_memory=False)
df_persons = pd.read_csv(PERSONS_URL, low_memory=False)

print("✅ Loaded Raw Data.")
print(f"   - Crashes: {df_crashes.shape}")
print(f"   - Persons: {df_persons.shape}")

# 2. Remove duplicates [cite: 53]
before = len(df_crashes)
df_crashes.drop_duplicates(inplace=True)
print(f"🧹 Removed {before - len(df_crashes)} duplicate crash rows.")

# 3. Handle missing Collision ID
# We drop rows without an ID because we cannot join them to the Persons dataset later.
df_crashes.dropna(subset=["COLLISION_ID"], inplace=True)

# 4. Parse date/time [cite: 52]
print("⏳ Parsing dates...")
# Combining Date and Time to create a proper Datetime object
df_crashes["CRASH_DATETIME"] = pd.to_datetime(
    df_crashes["CRASH DATE"] + " " + df_crashes["CRASH TIME"],
    errors="coerce"
)

# Extracting temporal features for visualization
df_crashes["CRASH_YEAR"] = df_crashes["CRASH_DATETIME"].dt.year
df_crashes["CRASH_MONTH"] = df_crashes["CRASH_DATETIME"].dt.month_name()
df_crashes["CRASH_DOW"] = df_crashes["CRASH_DATETIME"].dt.day_name()
df_crashes["CRASH_HOUR"] = df_crashes["CRASH_DATETIME"].dt.hour

# 5. Missing value report [cite: 48]
print("\n🔎 Top Missing Values (%):")
display((df_crashes.isna().mean() * 100).sort_values(ascending=False).head(10))

print("✅ PART 1 COMPLETE.")

👤 MEMBER 1: Loading datasets...


In [ ]:
# ==============================================================================
# Roba : Crash Dataset Cleaning & Standardization
# Responsibilities: Missing Values, IQR Outliers, Standardization, CLEANING FACTORS
# ==============================================================================

print("👤 MEMBER 2: Starting Cleaning & Outlier Detection...")

# 1. Standardize Boroughs [cite: 52]
def standardize_borough(v):
    if pd.isna(v): return np.nan
    v = str(v).upper().strip()
    mapping = {
        "BKLYN": "BROOKLYN", "KINGS": "BROOKLYN",
        "SI": "STATEN ISLAND", "NY": "MANHATTAN", "N.Y.": "MANHATTAN",
        "BRONX": "BRONX", "QUEENS": "QUEENS"
    }
    return mapping.get(v, v)

if "BOROUGH" in df_crashes.columns:
    df_crashes["BOROUGH"] = df_crashes["BOROUGH"].apply(standardize_borough)

    # DECISION: Impute missing Boroughs with Mode
    # JUSTIFICATION: Dropping rows with missing Boroughs would lose too much data (often >30%).
    mode_boro = df_crashes["BOROUGH"].mode()[0]
    df_crashes["BOROUGH"].fillna(mode_boro, inplace=True)
    print(f"   - JUSTIFICATION: Imputed missing Boroughs with Mode ('{mode_boro}') to preserve data volume[cite: 50].")

# 2. Numeric casting for injuries/killed
numeric_cols = ["NUMBER OF PERSONS INJURED", "NUMBER OF PERSONS KILLED"]
for col in numeric_cols:
    if col in df_crashes.columns:
        df_crashes[col] = pd.to_numeric(df_crashes[col], errors="coerce").fillna(0)

# 3. Outlier detection (IQR Method) [cite: 51]
if "NUMBER OF PERSONS INJURED" in df_crashes.columns:
    Q1 = df_crashes["NUMBER OF PERSONS INJURED"].quantile(0.25)
    Q3 = df_crashes["NUMBER OF PERSONS INJURED"].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + (1.5 * IQR)
    outliers = df_crashes[df_crashes["NUMBER OF PERSONS INJURED"] > upper_bound]

    # JUSTIFICATION: We detect outliers but DO NOT remove them.
    # High injury crashes are rare but critical for safety analysis.
    print(f"   - Outliers Detected: {len(outliers)} found > {upper_bound} injuries.")
    print("   - DECISION: Outliers kept. High casualty events are valid edge cases for this domain.")

# 4. Clean Contributing Factor (UPDATED FOR MESSY NUMBERS)
def clean_factor(v):
    if pd.isna(v): return "UNSPECIFIED"
    v = str(v).upper().strip()

    # FIX: Remove purely numeric entries (e.g., "1", "80") or messy codes
    # If the string is just digits, return UNSPECIFIED
    if v.replace('.', '', 1).isdigit():
        return "UNSPECIFIED"

    if v in ["", "NAN", "UNKNOWN", "UNSPECIFIED", "NULL", "OTHER"]:
        return "UNSPECIFIED"

    return v

fcol = "CONTRIBUTING FACTOR VEHICLE 1"
if fcol in df_crashes.columns:
    df_crashes[fcol] = df_crashes[fcol].apply(clean_factor)
    print(f"   - Cleaned '{fcol}': Removed numeric garbage (1, 80) and standardized text.")

# 5. Clean Vehicle Types
def clean_vehicle_type(v):
    if pd.isna(v): return "UNSPECIFIED"
    v = str(v).upper().strip()

    if v in ["N/A", "NONE", "", "UNKNOWN", "UNSPECIFIED"]: return "UNSPECIFIED"

    # Grouping logic to reduce cardinality
    if any(x in v for x in ["SEDAN", "PASSENGER", "CAR"]): return "SEDAN"
    if any(x in v for x in ["SUV", "SPORT UTILITY", "4X4", "WAGON"]): return "SUV"
    if any(x in v for x in ["TRUCK", "PICK"]): return "TRUCK"
    if "TAXI" in v or "LIVERY" in v: return "TAXI"
    if "BUS" in v: return "BUS"
    if any(x in v for x in ["MOTORCYCLE", "SCOOTER", "MOPED"]): return "MOTORCYCLE"
    if any(x in v for x in ["POLICE", "FIRE", "AMBULANCE"]): return "EMERGENCY VEHICLE"
    if "BICYCLE" in v: return "BICYCLE"
    return "OTHER"

if "VEHICLE TYPE CODE 1" in df_crashes.columns:
    df_crashes["VEHICLE TYPE CODE 1"] = df_crashes["VEHICLE TYPE CODE 1"].apply(clean_vehicle_type)

print("✅ PART 2 COMPLETE.")

In [ ]:
# ==============================================================================
# Sara: Integration With Persons Dataset
# Responsibilities: Merging Datasets, Handling Post-Integration Nulls
# ==============================================================================

print("👤 MEMBER 3: Integrating Datasets...")

# Ensure Join Key is consistent
df_persons["COLLISION_ID"] = pd.to_numeric(df_persons["COLLISION_ID"], errors="coerce")

# 1. Aggregate Persons dataset [cite: 54]
# We aggregate BEFORE merging to avoid duplicating crash rows for every person involved.
# This creates a 1-to-1 relationship potential.
person_counts = (
    df_persons.groupby("COLLISION_ID").size().reset_index(name="PERSONS_RECORDED_DB")
)

# 2. Merge with Crashes dataset
# DECISION: Left Join used. We want to keep ALL crash data, even if no detailed person data exists.
df_integrated = df_crashes.merge(person_counts, on="COLLISION_ID", how="left")

# 3. Post-Integration Cleaning
# The merge created NaN for 'PERSONS_RECORDED_DB' where no match was found.
# We fill these with 0 because it implies zero person records were found for that crash.
df_integrated["PERSONS_RECORDED_DB"].fillna(0, inplace=True)

# 4. Category optimization (Memory usage)
cat_cols = ["BOROUGH", "CRASH_DOW", "CONTRIBUTING FACTOR VEHICLE 1", "VEHICLE TYPE CODE 1"]
for col in cat_cols:
    if col in df_integrated.columns:
        df_integrated[col] = df_integrated[col].astype("category")

print(f"✅ PART 3 COMPLETE. Integrated dataset shape: {df_integrated.shape}")

In [ ]:
# ==============================================================================
# Omar 4: EDA & Export
# Responsibilities: 10 Visualizations, Final CSV Export
# ==============================================================================

print("📊 MEMBER 4: Generating Visualizations & Exporting...")

# ---- Q1: Crashes per Borough (Bar) ----
if "BOROUGH" in df_integrated.columns:
    boro_counts = df_integrated["BOROUGH"].value_counts().reset_index()
    boro_counts.columns = ["BOROUGH", "COUNT"]
    fig1 = px.bar(boro_counts, x="BOROUGH", y="COUNT", title="Q1: Crashes by Borough")
    fig1.show()

# ---- Q2: Crash Density Heatmap (Day vs Hour) ----
if {"CRASH_DOW", "CRASH_HOUR"}.issubset(df_integrated.columns):
    heat = df_integrated.groupby(["CRASH_DOW", "CRASH_HOUR"]).size().reset_index(name="Count")
    order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    fig2 = px.density_heatmap(
        heat, x="CRASH_HOUR", y="CRASH_DOW", z="Count",
        category_orders={"CRASH_DOW": order},
        title="Q2: Crash Density Heatmap (Time Patterns)"
    )
    fig2.show()

# ---- Q3: Top Contributing Factors (Bar) ----
fcol = "CONTRIBUTING FACTOR VEHICLE 1"
if fcol in df_integrated.columns:
    fac = df_integrated[df_integrated[fcol] != "UNSPECIFIED"][fcol].value_counts().head(10)
    fig3 = px.bar(fac, title="Q3: Top 10 Contributing Factors (Excluding Unspecified)")
    fig3.show()

# ---- Q4: Vehicle Types (Pie) ----
vcol = "VEHICLE TYPE CODE 1"
if vcol in df_integrated.columns:
    veh = df_integrated[vcol].value_counts().head(10).reset_index()
    veh.columns = ["Vehicle Type", "Count"]
    fig4 = px.pie(veh, names="Vehicle Type", values="Count", title="Q4: Top Vehicle Types Involved")
    fig4.show()

# ---- Q5: Monthly Trend (Bar) ----
if "CRASH_MONTH" in df_integrated.columns:
    monthly = df_integrated["CRASH_MONTH"].value_counts()
    fig5 = px.bar(monthly, title="Q5: Monthly Crash Seasonality")
    fig5.show()

# ---- Q6: Geographic Distribution (Map) ----
if {"LATITUDE", "LONGITUDE"}.issubset(df_integrated.columns):
    sample = df_integrated.dropna(subset=["LATITUDE", "LONGITUDE"]).sample(n=1000, random_state=42)
    fig6 = px.scatter_mapbox(
        sample, lat="LATITUDE", lon="LONGITUDE",
        color="BOROUGH", zoom=9,
        mapbox_style="open-street-map",
        title="Q6: Spatial Distribution (1000 Sampled Crashes)"
    )
    fig6.show()

# ---- Q7: Fatalities Weekend vs Weekday (Pie) ----
if "CRASH_DOW" in df_integrated.columns:
    df_integrated["IS_WEEKEND"] = df_integrated["CRASH_DOW"].isin(["Saturday", "Sunday"])
    fatal = df_integrated.groupby("IS_WEEKEND")["NUMBER OF PERSONS KILLED"].sum().reset_index()
    fatal["Type"] = fatal["IS_WEEKEND"].map({True: "Weekend", False: "Weekday"})
    fig7 = px.pie(fatal, values="NUMBER OF PERSONS KILLED", names="Type", title="Q7: Share of Fatalities (Weekend vs Weekday)")
    fig7.show()

# ---- Q8: Injury Distribution (Box) ----
if {"BOROUGH", "NUMBER OF PERSONS INJURED"}.issubset(df_integrated.columns):
    fig8 = px.box(df_integrated, x="BOROUGH", y="NUMBER OF PERSONS INJURED", title="Q8: Distribution of Injuries per Borough")
    fig8.update_yaxes(range=[0, 10])
    fig8.show()

# ---- Q9: Yearly Trend (Line) ----
if "CRASH_YEAR" in df_integrated.columns:
    yearly = df_integrated.groupby("CRASH_YEAR")["NUMBER OF PERSONS INJURED"].sum().reset_index()
    fig9 = px.line(yearly, x="CRASH_YEAR", y="NUMBER OF PERSONS INJURED", title="Q9: Total Injuries per Year (Trend)", markers=True)
    fig9.show()

# ---- Q10: Pedestrians vs Motorists (Comparison) ----
ped = "NUMBER OF PEDESTRIANS INJURED"
mot = "NUMBER OF MOTORIST INJURED"
if ped in df_integrated.columns and mot in df_integrated.columns:
    victims = df_integrated[[ped, mot]].sum().reset_index()
    victims.columns = ["Victim Type", "Total Injured"]
    fig10 = px.bar(victims, x="Victim Type", y="Total Injured", title="Q10: Pedestrians vs Motorists Injured")
    fig10.show()


# ⭐⭐⭐ SAFE COLUMN DROP (removes 4–7 MB, no effect on website) ⭐⭐⭐
safe_drop = [
    "ON STREET NAME",
    "CROSS STREET NAME",
    "OFF STREET NAME",
    "LOCATION"
]

df_integrated.drop(columns=[c for c in safe_drop if c in df_integrated.columns], inplace=True)


# ---- Export Final CSV ----
output = os.path.join(DATA_DIR, "nyc_crashes_final.csv")
df_integrated.to_csv(output, index=False)

print("💾 Final dataset saved:", output)
print("✅ PART 4 COMPLETE.")

In [ ]:
# ==============================================================================
# Omar : EDA & Export
# Responsibilities: 10 Visualizations, Final CSV Export
# ==============================================================================

print("📊 MEMBER 4: Generating Visualizations & Exporting...")

# ---- Q1: Crashes per Borough (Bar) ----
if "BOROUGH" in df_integrated.columns:
    boro_counts = df_integrated["BOROUGH"].value_counts().reset_index()
    boro_counts.columns = ["BOROUGH", "COUNT"]
    fig1 = px.bar(boro_counts, x="BOROUGH", y="COUNT", title="Q1: Crashes by Borough")
    fig1.show()

# ---- Q2: Crash Density Heatmap (Day vs Hour) ----
if {"CRASH_DOW", "CRASH_HOUR"}.issubset(df_integrated.columns):
    heat = df_integrated.groupby(["CRASH_DOW", "CRASH_HOUR"]).size().reset_index(name="Count")
    order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    fig2 = px.density_heatmap(
        heat, x="CRASH_HOUR", y="CRASH_DOW", z="Count",
        category_orders={"CRASH_DOW": order},
        title="Q2: Crash Density Heatmap (Time Patterns)"
    )
    fig2.show()

# ---- Q3: Top Contributing Factors (Bar) ----
fcol = "CONTRIBUTING FACTOR VEHICLE 1"
if fcol in df_integrated.columns:
    fac = df_integrated[df_integrated[fcol] != "UNSPECIFIED"][fcol].value_counts().head(10)
    fig3 = px.bar(fac, title="Q3: Top 10 Contributing Factors (Excluding Unspecified)")
    fig3.show()

# ---- Q4: Vehicle Types (Pie) ----
vcol = "VEHICLE TYPE CODE 1"
if vcol in df_integrated.columns:
    veh = df_integrated[vcol].value_counts().head(10).reset_index()
    veh.columns = ["Vehicle Type", "Count"]
    fig4 = px.pie(veh, names="Vehicle Type", values="Count", title="Q4: Top Vehicle Types Involved")
    fig4.show()

# ---- Q5: Monthly Trend (Bar) ----
if "CRASH_MONTH" in df_integrated.columns:
    monthly = df_integrated["CRASH_MONTH"].value_counts()
    fig5 = px.bar(monthly, title="Q5: Monthly Crash Seasonality")
    fig5.show()

# ---- Q6: Geographic Distribution (Map) ----
if {"LATITUDE", "LONGITUDE"}.issubset(df_integrated.columns):
    sample = df_integrated.dropna(subset=["LATITUDE", "LONGITUDE"]).sample(n=1000, random_state=42)
    fig6 = px.scatter_mapbox(
        sample, lat="LATITUDE", lon="LONGITUDE",
        color="BOROUGH", zoom=9,
        mapbox_style="open-street-map",
        title="Q6: Spatial Distribution (1000 Sampled Crashes)"
    )
    fig6.show()

# ---- Q7: Fatalities Weekend vs Weekday (Pie) ----
if "CRASH_DOW" in df_integrated.columns:
    df_integrated["IS_WEEKEND"] = df_integrated["CRASH_DOW"].isin(["Saturday", "Sunday"])
    fatal = df_integrated.groupby("IS_WEEKEND")["NUMBER OF PERSONS KILLED"].sum().reset_index()
    fatal["Type"] = fatal["IS_WEEKEND"].map({True: "Weekend", False: "Weekday"})
    fig7 = px.pie(fatal, values="NUMBER OF PERSONS KILLED", names="Type", title="Q7: Share of Fatalities (Weekend vs Weekday)")
    fig7.show()

# ---- Q8: Injury Distribution (Box) ----
if {"BOROUGH", "NUMBER OF PERSONS INJURED"}.issubset(df_integrated.columns):
    fig8 = px.box(df_integrated, x="BOROUGH", y="NUMBER OF PERSONS INJURED", title="Q8: Distribution of Injuries per Borough")
    fig8.update_yaxes(range=[0, 10])
    fig8.show()

# ---- Q9: Yearly Trend (Line) ----
if "CRASH_YEAR" in df_integrated.columns:
    yearly = df_integrated.groupby("CRASH_YEAR")["NUMBER OF PERSONS INJURED"].sum().reset_index()
    fig9 = px.line(yearly, x="CRASH_YEAR", y="NUMBER OF PERSONS INJURED", title="Q9: Total Injuries per Year (Trend)", markers=True)
    fig9.show()

# ---- Q10: Pedestrians vs Motorists (Comparison) ----
ped = "NUMBER OF PEDESTRIANS INJURED"
mot = "NUMBER OF MOTORIST INJURED"
if ped in df_integrated.columns and mot in df_integrated.columns:
    victims = df_integrated[[ped, mot]].sum().reset_index()
    victims.columns = ["Victim Type", "Total Injured"]
    fig10 = px.bar(victims, x="Victim Type", y="Total Injured", title="Q10: Pedestrians vs Motorists Injured")
    fig10.show()



safe_drop = [
    "ON STREET NAME",
    "CROSS STREET NAME",
    "OFF STREET NAME",
    "LOCATION"
]

df_integrated.drop(columns=[c for c in safe_drop if c in df_integrated.columns], inplace=True)


# ---- Export Final CSV ----
output = os.path.join(DATA_DIR, "nyc_crashes_final.csv")
df_integrated.to_csv(output, index=False)

print("💾 Final dataset saved:", output)

In [ ]:
import shutil
from google.colab import files

# 1. Zip the file
print("⏳ Zipping file...")
shutil.make_archive('nyc_crashes_final', 'zip', 'data')

# 2. Download the Zip
print("⬇ Downloading zip...")
files.download('nyc_crashes_final.zip')

### 📝 **Project Milestone 1: NYC Motor Vehicle Collisions Analysis**
**Team Members, Contributions & Research Findings**

This notebook documents the full data engineering pipeline—from data acquisition and cleaning to integration and visualization—for the NYC Motor Vehicle Collisions dataset. Below is the breakdown of responsibilities and the key insights derived from our analysis.

---

### 👤 **Member 1: Farida** (Setup & Data Loading)
**Assigned Part:** Cell 1
**Role:** Initial Setup
I was responsible for getting the project started. My main focus was setting up the environment and loading the raw data correctly from the NYC Open Data API.
* **Work Done:**
    * Imported all necessary libraries (Pandas, Plotly).
    * [cite_start]Loaded the Crashes and Persons CSVs directly from the NYC Open Data URLs[cite: 27].
    * [cite_start]**Cleaning:** I removed duplicate rows to make sure we didn't have repeated crash records[cite: 53].
    * [cite_start]**Formatting:** I converted the `CRASH DATE` and `TIME` columns into a proper datetime format so we could analyze trends by year, month, and hour[cite: 52].
* **My Research Questions & Findings:**
    1.  **Q1:** Which borough has the highest crash frequency?
        * **Finding:** **Brooklyn** consistently records the highest number of crashes, followed closely by Queens. This is likely due to the high population density and traffic volume in these boroughs.
    2.  **Q2:** At what time of day and day of the week do crashes occur most often?
        * **Finding:** Crashes peak during **weekdays** (particularly Friday) during the **evening rush hour (4 PM – 6 PM)**, coinciding with maximum traffic congestion.

---

### 👤 **Member 2: Roba** (Cleaning & Standardization)
**Assigned Part:** Cell 2
**Role:** Data Cleaning
My task was to clean the specific columns and handle outliers before we merged the data.
* **Work Done:**
    * [cite_start]**Boroughs:** I fixed inconsistent names (changing "BKLYN" to "BROOKLYN") and filled in missing borough names using the most common value (Mode)[cite: 50].
    * [cite_start]**Outliers:** I used the IQR method to find outliers in the "Number of Persons Injured" column but kept them because they represent serious accidents[cite: 51].
    * **Factors:** I cleaned the `CONTRIBUTING FACTOR` column by removing garbage data like "1" or "80".
    * **Vehicles:** I cleaned the `VEHICLE TYPE` column by merging messy names (e.g., "4 dr sedan") into clear categories (e.g., "Sedan", "SUV") so the website filter works properly.
* **My Research Questions & Findings:**
    1.  **Q3:** What are the top contributing factors (causes) of accidents?
        * **Finding:** **Driver Inattention/Distraction** is by far the leading cause of accidents, followed by Failure to Yield Right-of-Way.
    2.  **Q4:** Which vehicle types are most frequently involved in crashes?
        * **Finding:** **Sedans** and **SUVs** account for the vast majority of crashes, which aligns with the composition of personal vehicles on NYC roads.

---

### 👤 **Member 3: Sara** (Data Integration)
**Assigned Part:** Cell 3
**Role:** Merging Datasets
I was responsible for joining the two datasets together and handling the issues that came up during the merge.
* **Work Done:**
    * **Aggregation:** Before merging, I grouped the Persons dataset by `COLLISION_ID` to count the people involved. [cite_start]This was necessary to avoid creating duplicate rows in the main dataset[cite: 54].
    * [cite_start]**Merging:** I merged the cleaned Crashes data with the aggregated Persons data via `COLLISION_ID`[cite: 18].
    * [cite_start]**Post-Cleaning:** I filled in the null values created by the merge (setting missing person counts to 0)[cite: 56].
    * **Optimization:** I converted text columns to "Category" types to make the dataset smaller and faster to process.
* **My Research Questions & Findings:**
    1.  **Q5:** Is there a seasonal trend in crash frequencies?
        * **Finding:** Crash volumes remain relatively consistent, but slight peaks are often observed in the **summer months (June/July)** and late year holidays.
    2.  **Q6:** Where are the crash hotspots located geographically?
        * **Finding:** High-density clusters are visible in **Midtown Manhattan** and major intersections in Downtown Brooklyn, as shown in our geospatial scatter map.

---

### 👤 **Member 4: Omar** (Visualization EDA & Optimization)
**Assigned Part:** Cell 4
**Role:** Visualizations & Export Optimization
I created the charts and graphs to answer our research questions and prepared the final file for the website.
* **Work Done:**
    * [cite_start]**Plotting:** I wrote the code for 10 different visualizations using Plotly Express (Bar charts, Heatmaps, Maps, etc.)[cite: 66].
    * **Exporting:** I saved the final, cleaned dataframe to `nyc_crashes_final.csv`.
    * **Optimization:** To ensure the website fits within GitHub's 100MB file limit, I removed heavy, unused text columns (`ON STREET NAME`, `CROSS STREET NAME`, `OFF STREET NAME`, and `LOCATION`) before exporting. This reduced file size without affecting the dashboard's accuracy.
* **My Research Questions & Findings:**
    1.  **Q7:** Are weekends disproportionately more deadly than weekdays?
        * **Finding:** While **weekdays have a higher total volume** of crashes, weekends often show a higher *percentage* of fatalities per crash, possibly due to higher speeds or alcohol involvement.
    2.  **Q8:** How does the distribution of injuries per crash vary across different Boroughs?
        * **Finding:** **Brooklyn** and **Queens** have the widest distribution of injuries, with more outlier events (high casualty crashes) compared to Manhattan.

---

### 👤 **Member 5: Abdelrahman** (Website & Reporting)
**Assigned Part:** Dash App (External File) & Cell 5
**Role:** Web Dev & Final Report
I took the clean data and built the interactive website required for the project. I also wrote the final conclusion in the notebook.
* **Work Done:**
    * **Website:** I built the dashboard using Dash. [cite_start]I added the **dropdown filters** (Borough, Vehicle Type) and the **Search Mode**[cite: 63, 64].
    * [cite_start]**Interactivity:** I implemented the **"Generate Report"** button that updates the graphs when clicked[cite: 65].
    * [cite_start]**Deployment:** I deployed the website online so it can be accessed publicly[cite: 68].
    * **Report:** I summarized the answers to all 10 research questions in this notebook.
* **My Research Questions & Findings:**
    1.  **Q9:** How have total injuries trended over the years (2012–2025)?
        * **Finding:** Injuries were relatively stable until a **sharp drop in 2020** (due to COVID-19 lockdowns), followed by a gradual return to pre-pandemic levels in recent years.
    2.  **Q10:** Who is injured more frequently: Pedestrians or Motorists?
        * **Finding:** **Motorists** suffer the highest raw number of injuries, but Pedestrians have a much higher vulnerability and fatality rate relative to the number of incidents.